## Creating Dataset for Machine Learning with SD Score

The below script creates a dataset for machine learning with the following features:

- TVL: Total Value Locked in USD
- APY: Annual Percentage Yield
- APY Mean 7D: The average APY over the last 7 days
- APY Std 7D: The standard deviation of the APY over the last 7 days
- TVL Percentile: The percentile of TVL at each date
- APY 7D Percentile: The percentile of the 7 day APY at each date
- APY 30D Percentile: The percentile of the 30 day APY at each date
- APY 7D Std Ratio: The average APY for 7 days divided by the standard deviation of the APY for 7 days
- TVL Change 7D: The change in TVL over the last 7 days
- TVL Change 1D: The change in TVL over the last 1 day
- SD Score: The product of '7 day APY percentile' and 'tvl percentile' multiplied by 100
- SD Score 7D Avg: The rolling mean of the past 7 days SD score
- SD Score 7D Std: The standard deviation of the SD_Score_7D over the last 7 days

The dataset is a derivtive and statiscial description of TVL and APY data for DeFi protocols. The source of the data is DeFiLlama



In [1]:
# Import libraries and dependencies
import pandas as pd

data = pd.read_csv(r'/Users/karolk/Python_Work/Data_Sets/Global_Data/DeFi_Global_DB.csv', index_col=0)
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 4)
pd.options.display.float_format = '{:,.2f}'.format


In [2]:
#set date and time added as datetime objects
data['date'] = pd.to_datetime(data['date'])
data['time added'] = pd.to_datetime(data['time added'])

#drop all pools that have a TVL of 0 or a APY of 0
data = data[(data['tvlUsd'] > 0) & (data['apy'] > 0)]

# specify the start and end date
analysis_date = pd.to_datetime('2023-12-31') # convert the end_date to a datetime object
# end date is 21 days after the analysis date
end_date = analysis_date + pd.DateOffset(days=10) 
# start date is 21 days before the analysis date
start_date = analysis_date - pd.DateOffset(days=21)
data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]


In [3]:

# select only stablecoin is true
#data = data[data['stablecoin'] == True]

# sort the data table by pool and by date
data = data.sort_values(['pool', 'date'], ascending=[True, True]).reset_index(drop=True)

# create a new column which would the average APY over the last 7 days using the 'apy' column
data['apyMean7d'] = data.groupby('pool')['apy'].transform(lambda x: x.rolling(7, 1).mean())

# create a new column for the standard deviation of the APY over the last 7 days
data['apyStd7d'] = data.groupby('pool')['apy'].transform(lambda x: x.rolling(7, 1).std())

# getting the forward mean APY for the next 7 days. 
data['apyMean7dForward'] = data.groupby('pool')['apy'].transform(lambda x: x.shift(-7).rolling(7, 1).mean())

# getting the APY in 7 days
data['apy7dForward'] = data.groupby('pool')['apy'].shift(-7)

# creating a column for change in APY over the last 7 days measured
data['apyChange7d'] = data.groupby(['pool'])['apy'].pct_change(7)*100

# creating a column for the change in APY over the last 7 days in percentage
data['apyChange7dPercent'] = data.groupby(['pool'])['apy'].pct_change(7) * 100

# create a column which has the percentile for TVL for the TVL at each date
data['tvlPercentile'] = data.groupby('date')['tvlUsd'].rank(pct=True)

# create a column which has the percentile for 7 day APY
data['apy7DPercentile'] = data.groupby('date')['apyMean7d'].rank(pct=True)

# creating 2 new columns which is the average APY for 7 days divided by the standard deviation of the APY for 7 days
data['apy7DStdRatio'] = data['apyMean7d'] / data['apyStd7d']

# calculate a new column for the change in TVL over the last 7 days
data['tvlChange7d'] = data.groupby(['pool'])['tvlUsd'].pct_change(periods=7) * 100

# creating column with 'SD_Score' which is the product of '7 day APY percentile' and 'tvl percentile' multiplied by 100
data['SD_Score'] = data['apy7DPercentile'] * data['tvlPercentile'] * 100

# creating a column 'SD_Score_7D' which is the rolling mean of the past 7 days SD score
data['SD_Score_7D_avg'] = data.groupby('pool')['SD_Score'].transform(lambda x: x.rolling(7, 1).mean())

# create a column 'SD_Score_7D_std' which is the standard deviation of the SD_Score_7D over the last 7 days
data['SD_Score_7D_std'] = data.groupby('pool')['SD_Score'].transform(lambda x: x.rolling(7, 1).std())

# create a new column for forward SD_Score_7D_avg
data['SD_Score_7D_forward_rolling'] = data.groupby('pool')['SD_Score'].transform(lambda x: x.shift(-7).rolling(7, 1).mean())

# create a new column for the forward SD_Score_7D which is the SD_Score 7 days in the future
data['SD_Score_7D_forward'] = data.groupby('pool')['SD_Score'].shift(-7)

# create a new column which is the change in SD_Score over the last 7 days
data['SD_Score_7D_change'] = data.groupby(['pool'])['SD_Score'].pct_change(7)*100

# create a new column which is the change in SD_Score over the last 7 days
data['SD_Score_7D_forward_change'] = data.groupby(['pool'])['SD_Score'].pct_change(-7)*100


In [4]:
# Data Transformation - Adding new columns for the underlying tokens & taking only the most recent date

# create a list of all unique symbols in the data set
symbols = data['symbol'].unique()

# sort the symbols alphabetically
symbols.sort()

# splitting the 'symbol' column into 4 new columns using str.split() method with '-' as the separator
symbol_split = data['symbol'].str.split('-', expand=True, n=3)

# adding the 4 new columns to the data dataframe
symbol_split.columns = ['token_id_1', 'token_id_2', 'token_id_3', 'token_id_4']
data = pd.concat([data, symbol_split], axis=1)

# create a new column called num_tokens which is the number of tokens in the symbol
data['num_tokens'] = data['symbol'].str.count('-') + 1


In [5]:
# select the data that is equal to or less than the analysis date
data = data[data['date'] <= analysis_date]

# select data that is 7 days after the start date
data = data[data['date'] >= start_date + pd.DateOffset(days=7)]

In [6]:
data

,chain,project,symbol,tvlUsd,apy,pool,stablecoin,ilRisk,exposure,outlier,apyMean30d,date,time added,new_upload,possible_error,apyMean7d,apyStd7d,apyMean7dForward,apy7dForward,apyChange7d,apyChange7dPercent,tvlPercentile,apy7DPercentile,apy7DStdRatio,tvlChange7d,SD_Score,SD_Score_7D_avg,SD_Score_7D_std,SD_Score_7D_forward_rolling,SD_Score_7D_forward,SD_Score_7D_change,SD_Score_7D_forward_change,token_id_1,token_id_2,token_id_3,token_id_4,num_tokens
6,Base,uniswap-v3,ISK-WETH,"113,970.00",2.85,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,3.46,2023-12-17,2023-12-17 07:03:11,False,False,3.11,1.49,2.49,4.64,NaN,NaN,0.49,0.25,2.09,NaN,12.24,10.95,0.96,11.68,11.54,NaN,6.05,ISK,WETH,None,None,2
7,Base,uniswap-v3,ISK-WETH,"113,706.00",0.58,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,3.42,2023-12-18,2023-12-18 07:21:59,False,False,2.96,1.70,3.07,4.64,-64.06,-64.06,0.49,0.25,1.74,-0.20,12.23,11.22,1.02,11.78,12.94,18.47,-5.55,ISK,WETH,None,None,2
8,Base,uniswap-v3,ISK-WETH,"112,150.00",1.65,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,3.39,2023-12-19,2023-12-19 07:01:17,False,False,2.85,1.77,3.50,4.64,-32.93,-32.93,0.49,0.25,1.61,3.19,12.34,11.53,0.98,12.10,14.57,20.96,-15.30,ISK,WETH,None,None,2
9,Base,uniswap-v3,ISK-WETH,"114,564.00",1.55,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,3.26,2023-12-20,2023-12-20 07:31:38,False,False,2.78,1.81,3.72,3.11,-23.51,-23.51,0.50,0.25,1.53,3.71,12.16,11.88,0.57,12.52,15.12,25.37,-19.59,ISK,WETH,None,None,2
10,Base,uniswap-v3,ISK-WETH,"117,558.00",1.52,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,3.20,2023-12-21,2023-12-21 08:56:46,False,False,2.38,1.72,4.41,6.38,-64.84,-64.84,0.50,0.22,1.38,4.79,11.22,11.90,0.54,13.30,16.64,1.33,-32.59,ISK,WETH,None,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313592,Solana,raydium,UNKNOWN-SOL,"18,394.00",3.83,fff61d0d-1ffd-43a8-b345-9743fbf3fb2a,False,yes,multi,True,150.13,2023-12-30,2023-12-30 07:01:24,False,False,4.32,1.29,1.28,0.18,7.58,7.58,0.19,0.30,3.35,-18.59,5.88,13.84,6.90,3.67,1.63,-67.33,259.68,UNKNOWN,SOL,None,None,2
313593,Solana,raydium,UNKNOWN-SOL,"18,789.00",0.21,fff61d0d-1ffd-43a8-b345-9743fbf3fb2a,False,yes,multi,True,137.17,2023-12-31,2023-12-31 07:01:19,False,False,3.84,2.03,1.26,0.02,-94.10,-94.10,0.19,0.28,1.89,-21.03,5.42,11.93,7.15,3.14,1.69,-71.08,220.21,UNKNOWN,SOL,None,None,2
313613,Solana,raydium,UNKNOWN-SOL,"14,202.00","1,956.01",fffc1e1e-0eeb-4f19-9617-b5b77822025e,False,yes,multi,True,"1,956.01",2023-12-29,2023-12-29 10:35:36,False,False,"1,956.01",NaN,NaN,NaN,NaN,NaN,0.13,0.96,NaN,NaN,12.69,12.69,NaN,NaN,NaN,NaN,NaN,UNKNOWN,SOL,None,None,2
313614,Solana,raydium,UNKNOWN-SOL,"14,202.00","1,956.01",fffc1e1e-0eeb-4f19-9617-b5b77822025e,False,yes,multi,True,"1,956.01",2023-12-30,2023-12-30 07:01:24,False,False,"1,956.01",0.00,NaN,NaN,NaN,NaN,0.14,0.96,inf,NaN,13.17,12.93,0.34,NaN,NaN,NaN,NaN,UNKNOWN,SOL,None,None,2


In [7]:
# save the data to a csv file
filepath= '/Users/karolk/Python_Work/ML_Price/Datasets/DeFi_Quant_Data.csv'
data.to_csv(filepath, index=False)